In [1]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.environ['OPENAI_API_KEY'] = ''

In [3]:
from langchain.chat_models import ChatOpenAI

In [4]:
llm = ChatOpenAI(temperature =0.0)

/Users/lucastavrs2/miniconda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
lang= 'inglês'
text= 'ainda'

In [6]:
prompt = f'''Traduza para o {lang} o seguinte texto:
{text}
'''

In [7]:
from langchain.prompts import ChatPromptTemplate
chat_template = ChatPromptTemplate.from_template(prompt)

In [8]:
chat_template

ChatPromptTemplate(input_variables=[], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Traduza para o inglês o seguinte texto:\nainda\n'))])

In [9]:
chat_template.messages[0].prompt.template

'Traduza para o inglês o seguinte texto:\nainda\n'

In [10]:
chat_template.messages[0].prompt.input_variables

[]

In [11]:
prompt_engineered = chat_template.format_messages(
                    lang = 'francês',
                    text = 'Leclerc se chateia com Ferrari: "Dificultando nossa própria vida.\
                    Piloto de Mônaco sugeriu que a equipe colocasse pneus secos para o Q2 na classificação do GP do Canadá,\
                    mas a escuderia negou; titular do carro 16 acabou eliminado e obteve 11º tempo da sessão.')

In [12]:
prompt_engineered

[HumanMessage(content='Traduza para o inglês o seguinte texto:\nainda\n')]

In [13]:
response = llm(prompt_engineered)

/Users/lucastavrs2/miniconda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [14]:
response.content

'still'

In [15]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [16]:
examples= [ 
    {
        "question": "Quem viveu mais, Muhammad Ali ou Alan Turing?",
        "answer":
    """
    São necessárias perguntas de acompanhamento aqui? Sim.
    Acompanhamento: Qual era a idade de Muhammad Ali quando ele morreu?
    Resposta intermediáia: Muhammad Ali tinha 74 anos quando ele morreu.
    Acompanhamento: Qual a idade de Alan Turing quando ele morreu?
    Resposta intermediária: Alan Turing tinha 41 anos de idade quando morreu.
    Portanto, a resposta final é: Muhammad Ali
    """
    },
    {
        "question": "Quando nasceu o fundador do craiglist?",
        "answer":
    """
    São necessárias perguntas complementares aqui? Sim.
    Acompanhamento: Quem foi o fundador da craiglist?
    Resposta intermediária: A craiglist foi fundada por Craig Newmark.
    Acompanhamento: Quando Carig Newmark nasceu?
    Resposta intermediária: Craig Newmark nasceu em 6 de dezembro de 1952
    Portanto, a resposta final é: 6 de dezembro de 1952    
    """
    }
]

In [17]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="pergunta: {question}\n{answer}")

In [18]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Pergunta: {input}",
    input_variables=["input"]
)

In [19]:
print(prompt.format(input="Quem ganhou mais prêmios Emmy, Mad Men ou Sopranos?"))

pergunta: Quem viveu mais, Muhammad Ali ou Alan Turing?

    São necessárias perguntas de acompanhamento aqui? Sim.
    Acompanhamento: Qual era a idade de Muhammad Ali quando ele morreu?
    Resposta intermediáia: Muhammad Ali tinha 74 anos quando ele morreu.
    Acompanhamento: Qual a idade de Alan Turing quando ele morreu?
    Resposta intermediária: Alan Turing tinha 41 anos de idade quando morreu.
    Portanto, a resposta final é: Muhammad Ali
    

pergunta: Quando nasceu o fundador do craiglist?

    São necessárias perguntas complementares aqui? Sim.
    Acompanhamento: Quem foi o fundador da craiglist?
    Resposta intermediária: A craiglist foi fundada por Craig Newmark.
    Acompanhamento: Quando Carig Newmark nasceu?
    Resposta intermediária: Craig Newmark nasceu em 6 de dezembro de 1952
    Portanto, a resposta final é: 6 de dezembro de 1952    
    

Pergunta: Quem ganhou mais prêmios Emmy, Mad Men ou Sopranos?


In [20]:
prompt = ChatPromptTemplate.from_template(prompt.format(input="Quem ganhou mais prêmios Emmy, Mad men ou Sopranpos?"))

In [21]:
result = llm(prompt.format_messages())

In [22]:
print(result.content)

São necessárias perguntas de acompanhamento aqui? Sim.
Acompanhamento: Quantos prêmios Emmy Mad Men ganhou?
Resposta intermediária: Mad Men ganhou um total de 16 prêmios Emmy.
Acompanhamento: Quantos prêmios Emmy Sopranos ganhou?
Resposta intermediária: Sopranos ganhou um total de 21 prêmios Emmy.
Portanto, a resposta final é: Sopranos.


In [23]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

In [24]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=55,
)

In [25]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Pergunta: {input}",
    input_variables=["input"]
)

In [26]:
print(prompt.format(input="Quem ganhou mais prêmios Emmy, Mad Men ou Sopranos?"))

Pergunta: Quem ganhou mais prêmios Emmy, Mad Men ou Sopranos?


In [27]:
desired_output= {
    'is_offensive': False,
    'main_point': 'Harry is a great youtuber'
}

In [28]:
template_with_output_format = '''
Para o seguinte comentário de um vídeo do youtube, extraia as seguintes informações\
is_offensive: Booleano - Se o comentário apresenta conteúdo ofensivo\
main_point: String - O ponto principal do comentário, em cinco palavras ou menos.\

Escreva a resposta em formato JSON com as seguintes propriedades: is_offensive, main_point.\

Comentário: {comment}
'''

In [29]:
comment = '''
mercedesw211
Há 2 horas
You know that Harry is a serious man who cares for quality content when he doesn't use jeremy for clickbait.
I really love this channel, no childish bs.
'''

In [30]:
chat_template = ChatPromptTemplate.from_template(template_with_output_format)

In [31]:
prompt_engineered = chat_template.format_messages(comment = comment)

In [32]:
request = llm(prompt_engineered)

In [33]:
request.content

'{\n  "is_offensive": false,\n  "main_point": "Harry cares for quality content"\n}'

In [34]:
type(request.content)

str

In [35]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [36]:
offensive = ResponseSchema(name="offensive", description="O comentário apresenta conteúdo ofensivo?.")
main_point = ResponseSchema(name="main_point", description="Argumento central do comentário")

response_schemas = [offensive, main_point]

In [37]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [38]:
format_instructions = output_parser.get_format_instructions()

In [39]:
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"offensive": string  // O comentário apresenta conteúdo ofensivo?.\n\t"main_point": string  // Argumento central do comentário\n}\n```'

In [40]:
template = '''
Para o seguinte comentário de um vídeo do youtube, extraia as seguintes informações\
is_offensive: booleano - Se o comentário apresenta conteúdo ofensivo\
main_point: String - O ponto principal do comentário, em cinco palavras ou menos\

{format_instructions}

Comentário: {comment}
'''

In [41]:
prompt = ChatPromptTemplate.from_template(template=template)

messages = prompt.format_messages(comment=comment, format_instructions=format_instructions)

In [42]:
print(messages[0].content)


Para o seguinte comentário de um vídeo do youtube, extraia as seguintes informaçõesis_offensive: booleano - Se o comentário apresenta conteúdo ofensivomain_point: String - O ponto principal do comentário, em cinco palavras ou menos
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"offensive": string  // O comentário apresenta conteúdo ofensivo?.
	"main_point": string  // Argumento central do comentário
}
```

Comentário: 
mercedesw211
Há 2 horas
You know that Harry is a serious man who cares for quality content when he doesn't use jeremy for clickbait.
I really love this channel, no childish bs.




In [43]:
response = llm(messages)

In [44]:
response.content

'```json\n{\n\t"offensive": "false",\n\t"main_point": "Harry cares for quality content"\n}\n```'

In [45]:
output_dict = output_parser.parse(response.content)

In [46]:
output_dict

{'offensive': 'false', 'main_point': 'Harry cares for quality content'}

In [47]:
output_dict.get('main_point')

'Harry cares for quality content'

In [48]:
type(output_dict)

dict

In [49]:
from langchain import LLMChain

In [50]:
prompt = ChatPromptTemplate.from_template(
    "Crie uma lista com as atividades necessárias para produzir o seguinte produto: {product}"
)

In [51]:
chain = LLMChain(llm=llm, prompt=prompt)

In [52]:
result = chain.run(product= 'Guitarra')
result

/Users/lucastavrs2/miniconda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'1. Pesquisar e selecionar os materiais necessários para a construção da guitarra, como madeira, metal, plástico, entre outros.\n2. Cortar e moldar a madeira de acordo com o design da guitarra.\n3. Montar o corpo da guitarra, incluindo a instalação do braço, captadores, ponte e tarraxas.\n4. Lixar e polir a guitarra para garantir um acabamento suave e brilhante.\n5. Pintar ou envernizar a guitarra de acordo com a preferência do cliente.\n6. Instalar os controles de volume, tonalidade e chave seletora.\n7. Testar a guitarra para garantir que todos os componentes estejam funcionando corretamente.\n8. Embalar a guitarra de forma segura para o transporte e entrega ao cliente.'

In [53]:
products = [{'product': 'Frigideira'}, {'product': 'Cafeteira' }]

In [54]:
result = chain.apply(products)
result

[{'text': '1. Pesquisar e selecionar os materiais adequados para a frigideira, como aço inoxidável, alumínio ou ferro fundido.\n2. Projetar o design da frigideira, considerando o tamanho, formato e espessura do material.\n3. Comprar os materiais necessários para a produção da frigideira.\n4. Cortar e moldar o material de acordo com o design escolhido.\n5. Soldar ou fundir as partes da frigideira, se necessário.\n6. Polir e lixar a superfície da frigideira para garantir um acabamento liso.\n7. Adicionar revestimentos antiaderentes, se desejado.\n8. Testar a frigideira para garantir que ela está pronta para uso.\n9. Embalar a frigideira de forma segura e adequada para o transporte.\n10. Distribuir a frigideira para os pontos de venda ou clientes finais.'},
 {'text': '1. Pesquisar e selecionar os materiais necessários para a fabricação da cafeteira (aço inoxidável, plástico resistente ao calor, vidro, etc.)\n2. Projetar o design da cafeteira, levando em consideração a funcionalidade e est

In [55]:
from langchain.chains import SimpleSequentialChain

In [56]:
# Primeiro prompt
first_prompt = ChatPromptTemplate.from_template("Crie uma lista com as atividades necessárias para produzir o seguinte produto:{product}?")

# Chain 1
list_activities = LLMChain(llm = llm, prompt = first_prompt)


In [57]:
# Segundo prompt
second_prompt = ChatPromptTemplate.from_template("Dentre as atividades escreva qual delas é a mais importante e o motivo:{list}")

# Chain 2
most_important_activities = LLMChain(llm = llm, prompt = second_prompt)


In [58]:
combined_chain = SimpleSequentialChain(chains=[list_activities, most_important_activities], verbose=True)

In [59]:
result = combined_chain.run('Bootcamp de programação')
result



> Entering new SimpleSequentialChain chain...
1. Definir o objetivo e público-alvo do bootcamp de programação
2. Elaborar o conteúdo programático do bootcamp, incluindo linguagens de programação a serem ensinadas, metodologia de ensino, etc.
3. Contratar instrutores qualificados para ministrar as aulas
4. Definir a carga horária e a duração do bootcamp
5. Providenciar o local onde as aulas serão ministradas
6. Organizar a logística do evento, como alimentação, material didático, etc.
7. Divulgar o bootcamp para atrair os participantes
8. Realizar o processo de inscrição e seleção dos participantes
9. Realizar o bootcamp, acompanhando o desenvolvimento dos participantes e garantindo a qualidade do ensino
10. Avaliar o desempenho dos participantes e fornecer certificados de conclusão.
A atividade mais importante dentre as listadas é a número 1: Definir o objetivo e público-alvo do bootcamp de programação. Isso porque é fundamental ter clareza sobre o propósito do evento e para quem ele

'A atividade mais importante dentre as listadas é a número 1: Definir o objetivo e público-alvo do bootcamp de programação. Isso porque é fundamental ter clareza sobre o propósito do evento e para quem ele se destina, a fim de direcionar todas as outras etapas do planejamento. Ao definir o objetivo e o público-alvo, é possível elaborar um conteúdo programático adequado, contratar instrutores qualificados, definir a carga horária e duração do bootcamp, organizar a logística do evento, divulgar de forma eficaz, realizar o processo de inscrição e seleção dos participantes, acompanhar o desenvolvimento dos participantes durante o bootcamp e avaliar o desempenho ao final. Portanto, a definição do objetivo e público-alvo é a base para o sucesso de todo o processo.'

In [60]:
import langchain
langchain.degub = False

In [61]:
combined_chain.run('Bootcamp de programação')



> Entering new SimpleSequentialChain chain...
1. Definir o objetivo e público-alvo do bootcamp de programação
2. Elaborar o conteúdo programático do bootcamp, incluindo linguagens de programação a serem abordadas, metodologia de ensino, etc.
3. Contratar instrutores qualificados para ministrar as aulas
4. Definir a carga horária e a duração do bootcamp
5. Escolher o local onde as aulas serão ministradas
6. Providenciar os materiais necessários para as aulas, como computadores, projetores, etc.
7. Divulgar o bootcamp para atrair os participantes
8. Realizar o processo de inscrição e seleção dos participantes
9. Organizar a logística do evento, como alimentação, transporte, etc.
10. Avaliar o desempenho dos participantes ao longo do bootcamp
11. Emitir certificados de conclusão para os participantes
12. Realizar um evento de encerramento para celebrar o término do bootcamp.
A atividade mais importante dentre as listadas é a número 1: Definir o objetivo e público-alvo do bootcamp de pro

'A atividade mais importante dentre as listadas é a número 1: Definir o objetivo e público-alvo do bootcamp de programação. Isso porque é fundamental ter clareza sobre o propósito do bootcamp e para quem ele se destina, a fim de garantir que todas as outras etapas sejam planejadas de acordo com esses objetivos. A definição do público-alvo irá influenciar diretamente na escolha do conteúdo programático, na contratação dos instrutores, na divulgação do evento e na avaliação do desempenho dos participantes, entre outras atividades. Portanto, é essencial ter uma compreensão sólida do objetivo e do público-alvo para garantir o sucesso do bootcamp de programação.'

In [62]:
from langchain.chains import SequentialChain

In [63]:
# Determine o assunto da matéraise
first_prompt = ChatPromptTemplate.from_template("Escreva o nome da principal área do conhecimento que seguinte matéria aborda: \n\n{news}")

# Chain 1 : Entrada=Matéria de jornal; saída=Lista de tópicos
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="topic")



In [64]:
# Encontre pesquisadores relacionados
second_prompt = ChatPromptTemplate.from_template("Cite o nome de escritor famoso que escreve sobre a seguinte área do conhecimento: \n\n{topic}")

# Chain 2 : Entrada=Lista de tópicos; saída=Lista de pesquisadores
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="writer")



In [65]:
# Escreva um resumo da matéria
third_prompt = ChatPromptTemplate.from_template("Escreva um resumo da matéria a seguir: \n\n{news}")

# Chain 3 : Entrada=Matéria de jornal; saída=Resumo
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="summary")



In [66]:
# Escreva um resumo da matéria
fourth_prompt = ChatPromptTemplate.from_template("Reescreva o seguinte texto no estilo de {writer} \n\n{summary}")

# Chain 3 : Entrada=Matéria de jornal; saída=Resumo
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="writer_summary")



In [67]:
writer_news_chains = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["news"],
    output_variables=["topic", "writer", "summary", "writer_summary"],
    verbose=True
)

In [68]:
result = writer_news_chains(
    '''
        Quem é o milionário da tecnologia que tenta alcançar a juventude eterna e 'diminuir' sua idade em 28 anos
Da terapia com luz infravermelha, restrição a refeições após as 11h até o uso de anel peniano, Bryan Johnson recorre a uma série de ferramentas que ele considera rejuvenescedoras, mas são questionadas por especialistas.
Tomar mais de 100 comprimidos por dia, fazer a última refeição do dia às 11h (sim, da manhã) e usar um anel peniano enquanto dorme... Estas são algumas das táticas extravagantes adotadas pelo empresário Bryan Johnson, que, aos 46 anos, tem o plano de voltar a ter a saúde de um jovem de 18.

O milionário diz encarar seu corpo como se fosse um algoritmo, que pode ser configurado conforme os seus interesses. Para isso, criou que chama de protocolo Blueprint, que prevê, entre outras coisas, expor o rosto diariamente a um painel com luz infravermelha.
    '''
)

/Users/lucastavrs2/miniconda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


In [69]:
result

{'news': "\n        Quem é o milionário da tecnologia que tenta alcançar a juventude eterna e 'diminuir' sua idade em 28 anos\nDa terapia com luz infravermelha, restrição a refeições após as 11h até o uso de anel peniano, Bryan Johnson recorre a uma série de ferramentas que ele considera rejuvenescedoras, mas são questionadas por especialistas.\nTomar mais de 100 comprimidos por dia, fazer a última refeição do dia às 11h (sim, da manhã) e usar um anel peniano enquanto dorme... Estas são algumas das táticas extravagantes adotadas pelo empresário Bryan Johnson, que, aos 46 anos, tem o plano de voltar a ter a saúde de um jovem de 18.\n\nO milionário diz encarar seu corpo como se fosse um algoritmo, que pode ser configurado conforme os seus interesses. Para isso, criou que chama de protocolo Blueprint, que prevê, entre outras coisas, expor o rosto diariamente a um painel com luz infravermelha.\n    ",
 'topic': 'Medicina e Saúde',
 'writer': 'Oliver Sacks',
 'summary': 'O empresário Bryan 

In [70]:
chef_template = '''
Você é um chefe de cozinha renomado e com muita experiência, além de ser extremamente didático.
Se você não sabe a resposta de algo, você responde que não sabe.
Responda a seguinte questão: {input} 
'''

mechanic_template = '''
Você é um mecãnico com experiência em todos os tipos de automóveis.
Se você não sabe a resposta de algo, você responde que não sabe.
Responda a seguinte questão: {input} 
'''

In [71]:
prompt_infos= [
    {
        "name": "gastronomy",
        "description": "Recomendado para responder questões sobre gastronomia ",
        "prompt_template": chef_template
    },
    {
        "name": "auto mechanic",
        "description": "Recomendado para responder questões sobre mecânica de automóveis ",
        "prompt_template": mechanic_template
    }
]

In [72]:
from langchain.prompts import PromptTemplate
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

In [73]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [74]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [75]:
prompt_router_template = """
Dado um texto selecione o prompt mais adequado. \
Você receberá os nomes dos prompts disponíveis e uma decrição do motivo pelo qual o prompt é mais adequado. \
Você também pode revisar o texto original se achar que a revisão levará a uma resposta melhor do modelo de linguagem. \

<<<FORMATAÇÃO>>>
Retorne um trecho de código markdown com um objeto JSON formatado da seguinte maneira:
'''json
{{{{
"destination": string \ nome do prompt a ser usado ou "DEFAULT"
"next_inputs": string \ uma versão modificada da entrada original 
}}}}
''' 

LEMBRE-SE: "destination" DEVE ser um dos nomes de prompt candidatos especificados abaixo OU pode ser "DEFAULT" se a entrada não for adequada.
LEMBRE-SE: "next_inputs" pode ser apenas a entrada original se você achar que nenhuma modificação é necessária. 

<<<PROMPTS CANDIDATOS>>>
{destinations}

<<<INPUT>>>
{{input}}

<<<OUTPUT (lembre se de incluir o '''json)>>>
"""

In [76]:
router_template = prompt_router_template.format(
    destinations = destinations_str 
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt)

In [77]:
chain = MultiPromptChain(router_chain=router_chain,
                        destination_chains=destination_chains,
                        default_chain=default_chain, verbose= True    

)

In [78]:
result = chain.run("Qual é a receita de uma massa carbonara?")
result



> Entering new MultiPromptChain chain...


/Users/lucastavrs2/miniconda/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


gastronomy: {'input': 'Qual é a receita de uma massa carbonara?'}
> Finished chain.


'A receita tradicional de massa carbonara é feita com espaguete, bacon ou pancetta, queijo parmesão, ovos, pimenta do reino e sal. A preparação envolve cozinhar a massa al dente, fritar o bacon ou pancetta, misturar os ovos com queijo parmesão ralado, adicionar a massa cozida e misturar tudo rapidamente para que os ovos cozinhem com o calor da massa. Por fim, tempera-se com pimenta do reino e sal a gosto. Espero que essa explicação tenha sido útil!'

In [79]:
result = chain.run("Como posso trocar um pneu?")
result



> Entering new MultiPromptChain chain...


/Users/lucastavrs2/miniconda/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


auto mechanic: {'input': 'Como trocar um pneu de carro?'}
> Finished chain.


'Para trocar um pneu de carro, siga os seguintes passos:\n\n1. Estacione o carro em um local plano e seguro, acione o freio de mão e coloque o carro em ponto morto.\n2. Retire o estepe, o macaco e a chave de roda do porta-malas.\n3. Afrouxe os parafusos da roda com a chave de roda antes de levantar o carro.\n4. Posicione o macaco no ponto de apoio indicado no manual do carro e comece a levantar o veículo até que a roda fique suspensa.\n5. Retire os parafusos da roda e remova a roda furada.\n6. Coloque o estepe no lugar da roda furada e aperte os parafusos de forma cruzada.\n7. Abaixe o carro com cuidado, retire o macaco e termine de apertar os parafusos.\n8. Guarde a roda furada, o macaco e a chave de roda no porta-malas.\n\nLembre-se de sempre seguir as instruções do manual do proprietário do veículo e, se tiver dúvidas, procure a ajuda de um profissional.'

In [80]:
result = chain.run("Como posso criar um cacto?")
result



> Entering new MultiPromptChain chain...


/Users/lucastavrs2/miniconda/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


None: {'input': 'Como posso criar um cacto?'}
> Finished chain.


'Para criar um cacto, você precisará seguir alguns passos simples:\n\n1. Escolha um vaso adequado: Certifique-se de que o vaso tenha furos de drenagem para evitar o acúmulo de água, o que pode apodrecer as raízes do cacto.\n\n2. Escolha o substrato certo: Use um substrato específico para cactos e suculentas, que seja bem drenado e permita que as raízes respirem.\n\n3. Plante o cacto: Retire o cacto do vaso em que veio e plante-o no novo vaso, certificando-se de que as raízes estejam cobertas e firmes no substrato.\n\n4. Regue com moderação: Os cactos são plantas que não precisam de muita água, por isso regue-os apenas quando o substrato estiver seco. Evite regar em excesso, pois isso pode causar apodrecimento das raízes.\n\n5. Coloque em um local com luz solar: Os cactos precisam de bastante luz solar para crescer saudáveis. Coloque o vaso em um local com luz indireta ou direta por algumas horas por dia.\n\n6. Fertilize ocasionalmente: Você pode fertilizar o cacto uma vez por mês duran

In [81]:
from langchain.chains import TransformChain

In [82]:
def get_first_and_last_paragraph(inputs: dict) -> dict:
    text = inputs[ "text"]
    paragraphs = list(filter(None, text.split("\n\n")))
    shortened_text = paragraphs[0]
    shortened_text += paragraphs[-1]
    shortened_text = ''.join(shortened_text)
    return{"output_text": shortened_text}


transform_chain = TransformChain(
    input_variables=["text"], output_variables=["output_text"], transform=get_first_and_last_paragraph
)



In [83]:
template = """ 
Escreva um resumo so seguinte texto:
{output_text}

Resumo:

"""
prompt = PromptTemplate(input_variables=["output_text"], template=template)
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [84]:
sequential_chain = SimpleSequentialChain(chains=[transform_chain, llm_chain])

In [85]:
sequential_chain.run(
    '''
    Nova Iguaçu Futebol Clube é uma agremiação esportiva da cidade de Nova Iguaçu, no estado do Rio de Janeiro, no Brasil. Foi fundado a 1 de abril de 1990.
História

Sede do Nova Iguaçu Futebol Clube.
Foi concebido por 25 profissionais liberais encabeçados por Jânio Moraes, diretor-presidente do clube desde sua fundação até hoje. Teve seu início em uma sala de quarenta metros quadrados, localizada na Rua Topázio, número dez, no Centro de Nova Iguaçu. Um dos idealizadores do clube foi o ex-jogador Zinho, que foi titular da Seleção Brasileira na Copa do Mundo FIFA de 1994.
A cor adotada pelo time foi o laranja, em homenagem ao período em que Nova Iguaçu foi um dos maiores exportadores de laranjas do mundo, na década de 1930 (a fruta é, ainda hoje, um dos principais símbolos da cidade).

Time de Juniores em 2010, campeão do Torneio Otávio Pinto Guimarães de Juniores
Em 1991, os dirigentes do Nova Iguaçu obtiveram a posse de um terreno de 135.000 metros quadrados para a construção de seu Centro de Treinamento através do então secretário de Esportes do Governo Fernando Collor, o ex-jogador Zico.
Estreou no futebol profissional, em 1994, na Série B do Campeonato Estadual, na prática uma Terceira Divisão, pois havia acima o Módulo Intermediário, que, na realidade, era a verdadeira Segunda Divisão. Na primeira fase, se classificou em segundo, atrás somente do América de Três Rios. Na fase final, se habilitou à final contra o Goytacaz Futebol Clube, vencendo-o nos penâltis por 3 a 2, pois a partida terminou sem contagem no marcador. Foi o primeiro título da equipe comandada pelo técnico Paulo César.
    '''
)

'O Nova Iguaçu Futebol Clube é uma agremiação esportiva fundada em 1990 na cidade de Nova Iguaçu, no Rio de Janeiro. Em 1991, obteve um terreno para a construção de seu Centro de Treinamento com a ajuda do ex-jogador Zico. Estreou no futebol profissional em 1994, conquistando seu primeiro título na Série B do Campeonato Estadual.'

In [86]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [87]:
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=False
)

In [88]:
conversation.predict(input="Olá! Como vai você?")

'Olá! Estou bem, obrigado por perguntar. Como posso ajudá-lo hoje?'

In [89]:
conversation.predict(input="Como posso divulgar um curso online de programação?")

'Existem várias maneiras de divulgar um curso online de programação. Você pode usar plataformas de mídia social como Facebook, Instagram e Twitter para promover o curso. Além disso, você pode criar anúncios pagos no Google Ads ou em outras plataformas de publicidade online. Também é importante otimizar o SEO do seu site para que ele apareça nos resultados de pesquisa relevantes. Outra opção é colaborar com influenciadores ou blogs de tecnologia para alcançar um público maior. Espero que essas dicas sejam úteis para você!'

In [90]:
memory.buffer

'Human: Olá! Como vai você?\nAI: Olá! Estou bem, obrigado por perguntar. Como posso ajudá-lo hoje?\nHuman: Como posso divulgar um curso online de programação?\nAI: Existem várias maneiras de divulgar um curso online de programação. Você pode usar plataformas de mídia social como Facebook, Instagram e Twitter para promover o curso. Além disso, você pode criar anúncios pagos no Google Ads ou em outras plataformas de publicidade online. Também é importante otimizar o SEO do seu site para que ele apareça nos resultados de pesquisa relevantes. Outra opção é colaborar com influenciadores ou blogs de tecnologia para alcançar um público maior. Espero que essas dicas sejam úteis para você!'

In [91]:
memory.load_memory_variables({})

{'history': 'Human: Olá! Como vai você?\nAI: Olá! Estou bem, obrigado por perguntar. Como posso ajudá-lo hoje?\nHuman: Como posso divulgar um curso online de programação?\nAI: Existem várias maneiras de divulgar um curso online de programação. Você pode usar plataformas de mídia social como Facebook, Instagram e Twitter para promover o curso. Além disso, você pode criar anúncios pagos no Google Ads ou em outras plataformas de publicidade online. Também é importante otimizar o SEO do seu site para que ele apareça nos resultados de pesquisa relevantes. Outra opção é colaborar com influenciadores ou blogs de tecnologia para alcançar um público maior. Espero que essas dicas sejam úteis para você!'}

In [92]:
memory = ConversationBufferMemory()

In [93]:
memory.save_context(
    {"input": "Olá"},
    {"output": "Opa"}
)

In [8]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [95]:
result = embeddings.embed_query(" Olá! Tudo bem?")
result

[-0.010833039924406719,
 -0.00422142680610032,
 -0.0028657529155450535,
 -0.015069907593196458,
 -0.03453726335680424,
 0.01129625462921835,
 -0.01249443558533024,
 -0.008739311284050404,
 0.00933840129644506,
 -0.004783460424421012,
 0.02429713691345189,
 -0.03127623362306971,
 0.0018327848595599552,
 -0.025347089499300188,
 0.011765644934047683,
 -0.007448487014447155,
 0.023074250763301444,
 -0.007374372829315358,
 0.0028842813454126805,
 -0.0253964998873773,
 0.007843763599805983,
 -0.005993993977552182,
 0.00826992004789849,
 0.011889169041595302,
 -0.0021771076224985364,
 -0.011265372903839511,
 0.011395073542727412,
 -0.028608119233034714,
 0.03700773805717595,
 -0.02808931854012827,
 0.006392358362919862,
 0.010894801512519238,
 -0.006627053515334529,
 -0.00846138227489891,
 0.01351350737712228,
 -0.019739108241418987,
 -0.012834126182594255,
 -0.010061016347709914,
 0.013007059746896402,
 0.009733677881803888,
 0.0004709346122873893,
 -3.9662729413588646e-05,
 0.01637925959417

In [96]:
%pip install wikipedia
%pip install tiktoken

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [97]:
%pip install "langchain[docarray]"

Note: you may need to restart the kernel to use updated packages.


In [104]:
%pip show docarray

Name: docarray
Version: 0.32.1
Summary: The data structure for multimodal data
Home-page: https://docarray.jina.ai/
Author: DocArray
Author-email: 
License: Apache 2.0
Location: /Users/lucastavrs2/miniconda/lib/python3.10/site-packages
Requires: numpy, orjson, pydantic, rich, types-requests, typing-inspect
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [105]:
%pip install --upgrade docarray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 5.0 MB/s eta 0:00:0000:01
  Attempting uninstall: docarray
    Found existing installation: docarray 0.32.1
    Uninstalling docarray-0.32.1:
      Successfully uninstalled docarray-0.32.1
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain.document_loaders import WikipediaLoader

In [2]:
loader = WikipediaLoader(query="Seinfeld", load_max_docs=10, lang="pt")

In [3]:
docs = loader.load()

In [4]:
list(map(lambda x: x.metadata['title'], docs))

['Seinfeld',
 'Jerry Seinfeld',
 'Larry David',
 'Julia Louis-Dreyfus',
 'Evan Seinfeld',
 'Michael Richards',
 'Jerry Stiller',
 'Chris Rock',
 'Lista de episódios de Seinfeld',
 'Jason Alexander']

In [5]:
from langchain.vectorstores import DocArrayInMemorySearch

In [9]:
db = DocArrayInMemorySearch.from_documents(
    docs,
    embeddings
)

NameError: name 'embeddings' is not defined